<a href="https://colab.research.google.com/github/aghakishiyeva/Interpretable-ML/blob/main/AIPI590_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.graphics.gofplots import ProbPlot
from pygam import GAM, s, f
import statsmodels.api as sm

In [4]:
# Step 1: Load the data
url = 'https://raw.githubusercontent.com/aghakishiyeva/Interpretable-ML/main/Telco-Customer-Churn.csv'
df = pd.read_csv(url)

In [5]:
# Basic dataset overview
print("Dataset Overview:")
print(df.shape)
df.head()

Dataset Overview:
(7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [6]:
df.dtypes

,0
customerID,object
gender,object
SeniorCitizen,int64
Partner,object
Dependents,object
tenure,int64
PhoneService,object
MultipleLines,object
InternetService,object
OnlineSecurity,object


In [8]:
# Checking for missing values
print("\nMissing Values:")
print(df.isnull().sum())


Missing Values:
customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64


In [9]:
# Step 2: Data preprocessing
df['SeniorCitizen'] = df['SeniorCitizen'].astype('category')
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df.dropna(inplace=True)
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})
categorical_cols = df.select_dtypes(include=['object', 'category']).columns
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

In [ ]:
# Distribution of numerical variables
numerical_cols = df_encoded.select_dtypes(include=['float64', 'int64']).columns
fig, axes = plt.subplots(3, 3, figsize=(20, 20))
for i, col in enumerate(numerical_cols):
    sns.histplot(data=df_encoded, x=col, kde=True, ax=axes[i//3, i%3])
plt.tight_layout()
plt.show()

In [ ]:
# Step 4: Prepare data for modeling
X = df_encoded.drop('Churn', axis=1)
y = df_encoded['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Step 5: Check assumptions for each model type
def check_linear_regression_assumptions(X, y):
    X_const = sm.add_constant(X)
    model = sm.OLS(y, X_const).fit()

    # Linearity and Homoscedasticity
    plt.figure(figsize=(10, 6))
    plt.scatter(model.predict(), model.resid)
    plt.xlabel('Predicted values')
    plt.ylabel('Residuals')
    plt.title('Linearity and Homoscedasticity Check')
    plt.axhline(y=0, color='r', linestyle='--')
    plt.show()

    # Normality of residuals
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    sns.histplot(model.resid, kde=True, ax=ax1)
    ax1.set_title('Histogram of Residuals')
    stats.probplot(model.resid, dist="norm", plot=ax2)
    ax2.set_title('Q-Q Plot of Residuals')
    plt.show()

    # Multicollinearity
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    print("Variance Inflation Factors:")
    print(vif_data.sort_values('VIF', ascending=False))

def check_logistic_regression_assumptions(X, y):
    # Linearity of logit
    X_const = sm.add_constant(X)
    logit_model = sm.Logit(y, X_const)
    result = logit_model.fit()

    predicted_probs = result.predict(X_const)

    for feature in X.columns[:5]:  # Checking only first 5 features for brevity
        plt.figure(figsize=(10, 6))
        plt.scatter(X[feature], np.log(predicted_probs / (1 - predicted_probs)))
        plt.xlabel(feature)
        plt.ylabel('Log-odds')
        plt.title(f'Linearity of Logit: {feature}')
        plt.show()

    # No extreme outliers
    z_scores = np.abs((X - X.mean()) / X.std())
    print("Features with extreme outliers (|z-score| > 3):")
    print((z_scores > 3).sum())

def check_gam_assumptions(X, y):
    print("GAM assumptions will be checked via partial dependence plots after model fitting.")

In [ ]:
print("Checking Linear Regression Assumptions:")
check_linear_regression_assumptions(X_train_scaled, y_train)

print("\nChecking Logistic Regression Assumptions:")
check_logistic_regression_assumptions(X_train_scaled, y_train)

print("\nChecking GAM Assumptions:")
check_gam_assumptions(X_train[['tenure', 'MonthlyCharges', 'TotalCharges']], y_train)

In [ ]:
# Step 6: Build and evaluate models
# Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)
y_pred_lr = lr_model.predict(X_test_scaled)
print("\nLinear Regression Results:")
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred_lr)}")
print(f"R-squared: {r2_score(y_test, y_pred_lr)}")

# Logistic Regression
log_model = LogisticRegression(random_state=42)
log_model.fit(X_train_scaled, y_train)
y_pred_log = log_model.predict(X_test_scaled)
print("\nLogistic Regression Results:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_log)}")
print(classification_report(y_test, y_pred_log))

# GAM
gam = GAM(s(0) + s(1) + s(2))
X_gam = X_train[['tenure', 'MonthlyCharges', 'TotalCharges']]
gam.fit(X_gam, y_train)
y_pred_gam = gam.predict(X_test[['tenure', 'MonthlyCharges', 'TotalCharges']])
print("\nGAM Results:")
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred_gam)}")
print(f"R-squared: {r2_score(y_test, y_pred_gam)}")

In [ ]:
# Partial dependence plots for GAM
plt.figure(figsize=(15, 5))
for i, term in enumerate(gam.terms):
    if term.isintercept:
        continue
    XX = gam.generate_X_grid(term=i)
    pdep, confi = gam.partial_dependence(term=i, X=XX, width=0.95)
    plt.subplot(1, 3, i)
    plt.plot(XX[:, term.feature], pdep)
    plt.plot(XX[:, term.feature], confi, c='r', ls='--')
    plt.title(f'Partial Dependence ({X_gam.columns[i-1]})')
plt.tight_layout()
plt.show()

In [ ]:
# Step 7: Model Comparison and Recommendations
print("\nModel Comparison:")
print(f"Linear Regression R-squared: {r2_score(y_test, y_pred_lr)}")
print(f"Logistic Regression Accuracy: {accuracy_score(y_test, y_pred_log)}")
print(f"GAM R-squared: {r2_score(y_test, y_pred_gam)}")